### Drop upc, DD, FN / Create dummy variables for DD2 

In [1]:
train = pd.read_csv("data/train_v2.csv", index_col=0)

In [2]:
train.tail()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,DepartmentDescription_2,FinelineNumber
647049,39,191346,Sunday,3.239000e+10,1,PHARMACY OTC,pharmacy,1118.0
647050,39,191346,Sunday,7.874205e+09,1,FROZEN FOODS,food_1,1752.0
647051,39,191346,Sunday,4.072000e+03,1,PRODUCE,food_1,4170.0
647052,8,191347,Sunday,4.190008e+09,1,DAIRY,food_1,1512.0
647053,8,191347,Sunday,3.800060e+09,1,GROCERY DRY GOODS,food_1,3600.0


In [3]:
train['DepartmentDescription_2'].value_counts()

food_1           288582
household         52493
hygiene           46553
others            30102
pharmacy          26228
food_2            19179
deco              17916
baby              16697
hygene            14865
kids              14515
food_3            13669
men               12588
women             12557
service           11781
pets               9761
gardening          8162
kitchen            7600
office             6578
accessories        6220
accessaries        6099
electronics        5995
sport_outdoor      5947
auto               5414
entertainment      2985
interior           2784
home_garden        1720
tmp                  64
Name: DepartmentDescription_2, dtype: int64

In [4]:
# Weekday integer로..
def convert_weekday(day):
    if day == 'Sunday':
        return 0
    elif day == 'Monday':
        return 1
    elif day == 'Tuesday':
        return 2
    elif day == 'Wednesday':
        return 3
    elif day == 'Thursday':
        return 4
    elif day == 'Friday':
        return 5
    else:
        return 6
    
train.Weekday = train.Weekday.apply(convert_weekday)

In [5]:
# Returns column 추가
train['Returns'] = pd.Series([abs(num) if num < 0 else 0 for num in train.ScanCount], index=train.index)

In [6]:
# ScanCount rename Purchases
train.ScanCount = train.ScanCount.apply(lambda x: 0 if x < 0 else x)
train.rename(columns={'ScanCount': 'Purchases'}, inplace=True)

In [7]:
train.isnull().sum()

TripType                      0
VisitNumber                   0
Weekday                       0
Upc                        4129
Purchases                     0
DepartmentDescription      1361
DepartmentDescription_2       0
FinelineNumber             4129
Returns                       0
dtype: int64

In [8]:
train.drop(['Upc'], inplace=True, axis=1)
train['DepartmentDescription'] = train['DepartmentDescription'].fillna('None')
train['FinelineNumber'] = train['FinelineNumber'].fillna(-1)

In [9]:
dum_DD = pd.get_dummies(train.DepartmentDescription_2).astype(int, copy=False)
#dum_FN = pd.get_dummies(train.FinelineNumber).astype(int, copy=False)

In [10]:
train.drop(['DepartmentDescription', 'DepartmentDescription_2', 'FinelineNumber'], inplace=True, axis=1)

In [11]:
train.tail()

,TripType,VisitNumber,Weekday,Purchases,Returns
647049,39,191346,0,1,0
647050,39,191346,0,1,0
647051,39,191346,0,1,0
647052,8,191347,0,1,0
647053,8,191347,0,1,0


In [12]:
train.astype(int, copy=False)
train_2 = pd.concat([train, dum_DD], axis=1)

In [13]:
train_2.head()

,TripType,VisitNumber,Weekday,Purchases,Returns,accessaries,accessories,auto,baby,deco,...,kitchen,men,office,others,pets,pharmacy,service,sport_outdoor,tmp,women
0,999,5,5,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,30,7,5,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,30,7,5,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,26,8,5,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,26,8,5,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
train_2.columns

Index(['TripType', 'VisitNumber', 'Weekday', 'Purchases', 'Returns',
       'accessaries', 'accessories', 'auto', 'baby', 'deco', 'electronics',
       'entertainment', 'food_1', 'food_2', 'food_3', 'gardening',
       'home_garden', 'household', 'hygene', 'hygiene', 'interior', 'kids',
       'kitchen', 'men', 'office', 'others', 'pets', 'pharmacy', 'service',
       'sport_outdoor', 'tmp', 'women'],
      dtype='object')

In [15]:
len(train_2)

647054

In [17]:
def groupby_train(data):
    grouped = data.groupby("VisitNumber")
    grouped = grouped.agg({'Weekday': np.max, "TripType": np.max, 'Purchases': np.sum, 'Returns': np.sum,
                           'accessaries': np.sum, 'accessories': np.sum, 'auto': np.sum, 'baby': np.sum, 'deco': np.sum, 'electronics': np.sum,
       'entertainment': np.sum, 'food_1': np.sum, 'food_2': np.sum, 'food_3': np.sum, 'gardening': np.sum,
       'home_garden': np.sum, 'household': np.sum, 'hygene': np.sum, 'hygiene': np.sum, 'interior': np.sum, 'kids': np.sum,
       'kitchen': np.sum, 'men': np.sum, 'office': np.sum, 'others': np.sum, 'pets': np.sum, 'pharmacy': np.sum, 'service': np.sum,
       'sport_outdoor': np.sum, 'tmp': np.sum, 'women': np.sum})
    data = grouped[["TripType", "Weekday", "Purchases", "Returns",
                   'accessaries', 'accessories', 'auto', 'baby', 'deco', 'electronics',
       'entertainment', 'food_1', 'food_2', 'food_3', 'gardening',
       'home_garden', 'household', 'hygene', 'hygiene', 'interior', 'kids',
       'kitchen', 'men', 'office', 'others', 'pets', 'pharmacy', 'service',
       'sport_outdoor', 'tmp', 'women']]
    data['TripType'] = data['TripType'].map({3:0, 4:1, 5:2, 6:3, 7:4, 8:5, 9:6, 12:7, 14:8, 15:9,  
                                         18:10, 19:11, 20:12, 21:13, 22:14, 23:15, 24:16, 25:17, 26:18,  
                                         27:19, 28:20, 29:21, 30:22, 31:23, 32:24, 33:25, 34:26, 35:27, 
                                         36:28, 37:29, 38:30, 39:31, 40:32, 41:33, 42:34, 43:35, 44:36, 999:37})
    return data
       

In [18]:
train_3 = groupby_train(train_2)

In [39]:
train_3.head()

,TripType,Weekday,Purchases,Returns,accessaries,accessories,auto,baby,deco,electronics,...,kitchen,men,office,others,pets,pharmacy,service,sport_outdoor,tmp,women
VisitNumber,,,,,,,,,,,,,,,,,,,,,
5,37,5,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
7,22,5,2,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,18,5,30,2,0,0,0,0,0,0,...,0,0,0,1,2,0,0,0,0,0
9,5,5,3,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
10,5,5,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
len(train_3)

95674

In [21]:
train_3.to_csv("data/0409_train.csv")

In [22]:
def regroup_dep(DD):
    if DD == 'FINANCIAL SERVICES':
        return 'service'
    
    elif DD == 'SHOES':
        return 'accessaries'
    
    elif DD == 'PERSONAL CARE':
        return 'hygiene'
    
    elif DD == 'PAINT AND ACCESSORIES':
        return 'home_garden'
    
    elif DD == 'DSD GROCERY':
        return 'food_1'
    
    elif DD == 'MEAT - FRESH & FROZEN':
        return 'food_1'
    
    elif DD == 'DAIRY':
        return 'food_1'
    
    elif DD == 'PETS AND SUPPLIES':
        return 'pets'
    
    elif DD == 'HOUSEHOLD CHEMICALS/SUPP':
        return 'household'
    
    elif DD == 'IMPULSE MERCHANDISE':  # 간이 진열대의 상품에는 것은 여러가지가 있지만 (사탕, 음료수, 가그린, 잡지 등등)
        return 'others' # 이를 목적으로 방문하는 경우가 매우 적기 때문에 others로 분류를 해놓겠습니다.
    
    elif DD == 'PRODUCE': # 농장에서 재배된 신선식품을 뜻한다고 합니다.
        return 'food_1'
    
    elif DD == 'CANDY, TOBACCO, COOKIES':
        return 'food_3'
    
    elif DD == 'GROCERY DRY GOODS':
        return 'food_1'
    
    elif DD == 'BOYS WEAR':
        return 'kids'
    
    elif DD == 'FABRICS AND CRAFTS':
        return 'deco'
    
    elif DD == 'JEWELRY AND SUNGLASSES':
        return 'accessories'
    
    elif DD == 'MENS WEAR':
        return 'men'
    
    elif DD == 'ACCESSORIES':
        return 'accessories'
    
    elif DD == 'HOME MANAGEMENT':
        return 'household'
    
    elif DD == 'FROZEN FOODS':
        return 'food_1'
    
    elif DD == 'SERVICE DELI':
        return 'food_2'
    
    elif DD == 'INFANT CONSUMABLE HARDLINES':
        return 'baby'
    
    elif DD == 'PRE PACKED DELI':
        return 'food_2'
    
    elif DD == 'COOK AND DINE':
        return 'kitchen'
    
    elif DD == 'PHARMACY OTC':
        return 'pharmacy'
    
    elif DD == 'LADIESWEAR':
        return 'women'
    
    elif DD == 'COMM BREAD':
        return 'food_1'
    
    elif DD == 'BAKERY':
        return 'food_1'
    
    elif DD == 'HOUSEHOLD PAPER GOODS':
        return 'household'
    
    elif DD == 'CELEBRATION':
        return 'deco'
    
    elif DD == 'HARDWARE':
        return 'household'
    
    elif DD == 'BEAUTY':
        return 'hygene'
    
    elif DD == 'AUTOMOTIVE':
        return 'auto'
    
    elif DD == 'BOOKS AND MAGAZINES':
        return 'entertainment'
    
    elif DD == 'SEAFOOD':
        return 'food_1'
    
    elif DD == 'OFFICE SUPPLIES':
        return 'office'
    
    elif DD == 'LAWN AND GARDEN':
        return 'gardening'
    
    elif DD == 'SHEER HOSIERY':
        return 'women'
    
    elif DD == 'WIRELESS':
        return 'electronics'
    
    elif DD == 'BEDDING':
        return 'interior'
    
    elif DD == 'BATH AND SHOWER':
        return 'hygiene'
    
    elif DD == 'HORTICULTURE AND ACCESS':
        return 'gardening'
    
    elif DD == 'HOME DECOR':
        return 'deco'
    
    elif DD == 'TOYS':
        return 'kids'
    
    elif DD == 'INFANT APPAREL':
        return 'baby'
    
    elif DD == 'LADIES SOCKS':
        return 'women'
    
    elif DD == 'PLUS AND MATERNITY':
        return 'women'
    
    elif DD == 'ELECTRONICS':
        return 'electronics'
    
    elif DD == 'GIRLS WEAR, 4-6X  AND 7-14':
        return 'kids'
    
    elif DD == 'BRAS & SHAPEWEAR':
        return 'women'
    
    elif DD == 'LIQUOR,WINE,BEER':
        return 'food_3'
    
    elif DD == 'SLEEPWEAR/FOUNDATIONS':
        return 'accessories'
    
    elif DD == 'CAMERAS AND SUPPLIES':
        return 'electronics'
    
    elif DD == 'SPORTING GOODS':
        return 'sport_outdoor'
    
    elif DD == 'PLAYERS AND ELECTRONICS':
        return 'electronics'
    
    elif DD == 'PHARMACY RX':
        return 'pharmacy'
    
    elif DD == 'MENSWEAR':
        return 'men'
    
    elif DD == 'OPTICAL - FRAMES':
        return 'service'
    
    elif DD == 'SWIMWEAR/OUTERWEAR':
        return 'sport_outdoor'
    
    elif DD == 'OTHER DEPARTMENTS':
        return 'others'
    
    elif DD == 'MEDIA AND GAMING':
        return 'entertainment'
    
    elif DD == 'FURNITURE':
        return 'interior'
    
    elif DD == 'OPTICAL - LENSES':
        return 'service'
    
    elif DD == 'SEASONAL':
        return 'tmp'
    
    elif DD == 'LARGE HOUSEHOLD GOODS':
        return 'household'
    
    elif DD == '1-HR PHOTO':
        return 'service'
    
    elif DD == 'CONCEPT STORES':
        return 'tmp'
    
    elif DD == 'HEALTH AND BEAUTY AIDS':
        return 'hygiene'
    
    elif np.isnan(DD) == True:
        return 'others'

In [24]:
test = pd.read_csv("data/test.csv",)
test.head()

,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
0,1,Friday,7.250339e+10,1,SHOES,3002.0
1,1,Friday,1.707711e+09,1,DAIRY,1526.0
2,1,Friday,8.947000e+10,1,DAIRY,1431.0
3,1,Friday,8.849121e+10,1,GROCERY DRY GOODS,3555.0
4,2,Friday,2.840015e+09,1,DSD GROCERY,4408.0


In [25]:
test['DepartmentDescription_2'] = test['DepartmentDescription'].apply(regroup_dep)

In [26]:
test.head()

,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,DepartmentDescription_2
0,1,Friday,7.250339e+10,1,SHOES,3002.0,accessaries
1,1,Friday,1.707711e+09,1,DAIRY,1526.0,food_1
2,1,Friday,8.947000e+10,1,DAIRY,1431.0,food_1
3,1,Friday,8.849121e+10,1,GROCERY DRY GOODS,3555.0,food_1
4,2,Friday,2.840015e+09,1,DSD GROCERY,4408.0,food_1


In [27]:
# Weekday integer로..
def convert_weekday(day):
    if day == 'Sunday':
        return 0
    elif day == 'Monday':
        return 1
    elif day == 'Tuesday':
        return 2
    elif day == 'Wednesday':
        return 3
    elif day == 'Thursday':
        return 4
    elif day == 'Friday':
        return 5
    else:
        return 6
    
test.Weekday = test.Weekday.apply(convert_weekday)

In [28]:
# Returns column 추가
test['Returns'] = pd.Series([abs(num) if num < 0 else 0 for num in test.ScanCount], index=test.index)

In [29]:
# ScanCount rename Purchases
test.ScanCount = test.ScanCount.apply(lambda x: 0 if x < 0 else x)
test.rename(columns={'ScanCount': 'Purchases'}, inplace=True)

In [30]:
dum_DD_test = pd.get_dummies(test.DepartmentDescription_2).astype(int, copy=False)

In [31]:
test.drop(['Upc', 'DepartmentDescription', 'DepartmentDescription_2', 'FinelineNumber'], inplace=True, axis=1)

In [32]:
test.astype(int, copy=False)
test_2 = pd.concat([test, dum_DD_test], axis=1)

In [33]:
test_2.head()

,VisitNumber,Weekday,Purchases,Returns,accessaries,accessories,auto,baby,deco,electronics,...,kitchen,men,office,others,pets,pharmacy,service,sport_outdoor,tmp,women
0,1,5,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,5,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,5,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,5,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2,5,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
test_2.columns

Index(['VisitNumber', 'Weekday', 'Purchases', 'Returns', 'accessaries',
       'accessories', 'auto', 'baby', 'deco', 'electronics', 'entertainment',
       'food_1', 'food_2', 'food_3', 'gardening', 'home_garden', 'household',
       'hygene', 'hygiene', 'interior', 'kids', 'kitchen', 'men', 'office',
       'others', 'pets', 'pharmacy', 'service', 'sport_outdoor', 'tmp',
       'women'],
      dtype='object')

In [35]:
len(test_2)

653646

In [36]:
def groupby_test(data):
    grouped = data.groupby("VisitNumber")
    grouped = grouped.agg({'Weekday': np.max, 'Purchases': np.sum, 'Returns': np.sum,
                           'accessaries': np.sum, 'accessories': np.sum, 'auto': np.sum, 'baby': np.sum, 'deco': np.sum, 'electronics': np.sum,
       'entertainment': np.sum, 'food_1': np.sum, 'food_2': np.sum, 'food_3': np.sum, 'gardening': np.sum,
       'home_garden': np.sum, 'household': np.sum, 'hygene': np.sum, 'hygiene': np.sum, 'interior': np.sum, 'kids': np.sum,
       'kitchen': np.sum, 'men': np.sum, 'office': np.sum, 'others': np.sum, 'pets': np.sum, 'pharmacy': np.sum, 'service': np.sum,
       'sport_outdoor': np.sum, 'tmp': np.sum, 'women': np.sum})
    data = grouped[["Weekday", "Purchases", "Returns",
                   'accessaries', 'accessories', 'auto', 'baby', 'deco', 'electronics',
       'entertainment', 'food_1', 'food_2', 'food_3', 'gardening',
       'home_garden', 'household', 'hygene', 'hygiene', 'interior', 'kids',
       'kitchen', 'men', 'office', 'others', 'pets', 'pharmacy', 'service',
       'sport_outdoor', 'tmp', 'women']]
#     data['TripType'] = data['TripType'].map({3:0, 4:1, 5:2, 6:3, 7:4, 8:5, 9:6, 12:7, 14:8, 15:9,  
#                                          18:10, 19:11, 20:12, 21:13, 22:14, 23:15, 24:16, 25:17, 26:18,  
#                                          27:19, 28:20, 29:21, 30:22, 31:23, 32:24, 33:25, 34:26, 35:27, 
#                                          36:28, 37:29, 38:30, 39:31, 40:32, 41:33, 42:34, 43:35, 44:36, 999:37})
    return data
       

In [37]:
test_3 = groupby_test(test_2)

In [38]:
test_3.head()

,Weekday,Purchases,Returns,accessaries,accessories,auto,baby,deco,electronics,entertainment,...,kitchen,men,office,others,pets,pharmacy,service,sport_outdoor,tmp,women
VisitNumber,,,,,,,,,,,,,,,,,,,,,
1,5,4,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5,4,0,0,0,0,0,0,0,0,...,0,0,0,2,0,0,0,0,0,0
3,5,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
6,5,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
len(test_3)

95674

In [41]:
test_3.to_csv("data/0409_test.csv")